In [1]:
import pandas as pd
import math
import time
import numpy as np
from tqdm import tqdm 
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import datetime
from scipy.stats.mstats import winsorize

pd.options.mode.chained_assignment = None  # default='warn'

PATH = "C:/Users/jackl/OneDrive/Documents/finance_research/japan_qe/"

In [213]:
df = pd.read_pickle(PATH+'raw_data/capital_iq/missed_details.pkl')

In [215]:
#change columns
df.columns = df.iloc[0]

In [216]:
#remove empty columns
df = df.drop(columns=[None])

In [218]:
#remove header rows
df = df[~df['ticker'].str.contains('ticker')]

In [220]:
df['asof_date'] = pd.to_datetime(df['asof_date'])

In [221]:
df = df.sort_values(['sedol', 'asof_date'])

In [223]:
sedol_df_list = []
char_cols = ['ciq', 'market_cap', 'growth_value', 'high_yield', 'reported_style']

for sedol, sedol_df in tqdm(df.groupby('sedol')):     
    date_df_list = []
    counter = 0
    for date, date_df in sedol_df.groupby('asof_date'):
        if counter == 0: 
            chars_df = date_df[char_cols]
            chars_df = chars_df.reset_index(drop=True)
        change_df = date_df[['ticker', 'asof_date', 'sedol', 'comn', 'turnover_pct', 'turnover_category']]
        change_df = change_df.reset_index(drop=True)
        date_df_updated = pd.concat([change_df, chars_df], axis=1)
        date_df_list.append(date_df_updated)
        counter += 1
    sedol_df = pd.concat(date_df_list)
    sedol_df_list.append(sedol_df)
    
df = pd.concat(sedol_df_list)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  8.00it/s]


In [225]:
#gets rid of all useless rows
df = df[~df['ciq'].isna()]

In [227]:
#set nan types
df = df.astype(str)
df = df.replace(['None', '(Invalid Identifier)', 'nan', 'NaN', '0.0', '0'], math.nan)

In [229]:
df['asof_date'] = pd.to_datetime(df['asof_date'])
df['turnover_pct'] = df['turnover_pct'].astype(float)

In [230]:
df.to_pickle(PATH+'checkpoint_data/ownership_detailed_missed.pkl')

In [232]:
df1 = pd.read_pickle(PATH+'checkpoint_data/ownership_detailed_og.pkl')
df2 = pd.read_pickle(PATH+'checkpoint_data/ownership_detailed_missed.pkl')

In [233]:
df = pd.concat([df1, df2])

In [235]:
df.to_pickle(PATH+'checkpoint_data/ownership_detailed_complete.pkl')